# basics
## building dataset

In [1]:
from torch.utils.data import Dataset
from torch import Tensor
class MyDataset(Dataset):
    def __getitem__(self, index):
        return Tensor([index, 0.]), Tensor([3. * index + 4])
    def __len__(self):
        return 400
training_dataset = MyDataset()
print(len(training_dataset))
print(training_dataset[0])
print(training_dataset[1])

/home/hyperplane/anaconda3/envs/pytorch_test/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


400
(tensor([0., 0.]), tensor([4.]))
(tensor([1., 0.]), tensor([7.]))


## building dataloader

In [2]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(training_dataset, batch_size=4)
for x, y in train_dataloader:
    print(x)
    print(y)
    break

tensor([[0., 0.],
        [1., 0.],
        [2., 0.],
        [3., 0.]])
tensor([[ 4.],
        [ 7.],
        [10.],
        [13.]])


## building model

In [3]:
from torch import nn
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.model = nn.Linear(2, 1)
    def forward(self, x):
        return self.model(x)
model = MyModel()
print(model)

MyModel(
  (model): Linear(in_features=2, out_features=1, bias=True)
)


## building optimizer

In [4]:
from torch.optim import Adam
optimizer = Adam(model.parameters(), lr=1e-1)

## training

In [5]:
loss_fn = nn.MSELoss()
for epoch in range(100):
    for batch, (x, y) in enumerate(train_dataloader):
        loss = loss_fn(model(x), y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 20 == 19:
        print(f'epoch: {epoch}, loss: {loss.item()}')
        for n, p in model.named_parameters():
            print(f'name, parameters: {n, p}')
        print()

epoch: 19, loss: 9.914487600326538e-05
name, parameters: ('model.weight', Parameter containing:
tensor([[ 3.0004, -0.0428]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([3.8486], requires_grad=True))

epoch: 39, loss: 0.6594313979148865
name, parameters: ('model.weight', Parameter containing:
tensor([[ 3.0015, -0.0428]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([1.8757], requires_grad=True))

epoch: 59, loss: 0.0034229084849357605
name, parameters: ('model.weight', Parameter containing:
tensor([[ 3.0021, -0.0428]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([3.2428], requires_grad=True))

epoch: 79, loss: 49.69294738769531
name, parameters: ('model.weight', Parameter containing:
tensor([[ 3.0402, -0.0428]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([0.2279], requires_grad=True))

epoch: 99, loss: 0.025504864752292633
name, par

## testing

In [6]:
print(model(Tensor([100, 0])))
print(model(Tensor([10000000, 0])))

tensor([300.6250], grad_fn=<AddBackward0>)
tensor([30106792.], grad_fn=<AddBackward0>)


可以看到，这里泛化到`10000000`就有了不小的误差

## building lightning model

In [7]:
from pytorch_lightning import LightningModule
class MyLitModel(LightningModule):
    def __init__(self):
        super(MyLitModel, self).__init__()
        self.model = nn.Linear(2, 1)
    def forward(self, x):
        return self.model(x)
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=1e-1)
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        loss = loss_fn(self.model(x), y)
        self.log('train_loss', loss)
        return loss
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        loss = loss_fn(self.model(x), y)
        self.log('val_loss', loss)
model = MyLitModel()
print(model)

MyLitModel(
  (model): Linear(in_features=2, out_features=1, bias=True)
)


## building trainer

In [8]:
from pytorch_lightning import Trainer
trainer = Trainer(max_epochs=10)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/hyperplane/anaconda3/envs/pytorch_test/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1817: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=8)`.
  category=PossibleUserWarning,


## training using pytorch_lightning
这里训练和验证集是同一个

In [9]:
trainer.fit(model, train_dataloader, train_dataloader)


  | Name  | Type   | Params
---------------------------------
0 | model | Linear | 3     
---------------------------------
3         Trainable params
0         Non-trainable params
3         Total params
0.000     Total estimated model params size (MB)


/home/hyperplane/anaconda3/envs/pytorch_test/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 72 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/home/hyperplane/anaconda3/envs/pytorch_test/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 72 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Epoch 49: 100%|██████████| 200/200 [00:00<00:00, 259.73it/s, loss=0.00751, v_num=1] 
